In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import scsavailability as scs
    
import statsmodels.api as sm
from scipy import stats

from scsavailability import features as feat, model as md, plotting as pt, score as sc


2020-12-14 18:39:23,780 - NumExpr defaulting to 8 threads.


In [2]:
path = 'C:/Users/Jamie.williams/OneDrive - Newton Europe Ltd/Castle Donnington/Data/'

In [3]:
at = pd.read_csv(path+'active_totes_20201123.csv')
av = pd.read_csv(path+'Availability_with_Grey&Blue_1811-2511.csv')
#fa = pd.read_csv(path + 'scs_tote_matched.csv')
scs_raw = pd.read_csv(path + 'SCS_Alerts_NOV_new.csv')



In [4]:
fa = feat.add_code(scs_raw)
fa, unmapped = feat.add_tote_colour(fa)

Running: Function "load_tote_lookup" (id=2365752384240) was called 1 times


In [5]:
at = feat.pre_process_AT(at)

Running: Function "pre_process_AT" (id=2365752384672) was called 1 times


In [6]:
av = feat.pre_process_av(av)

Running: Function "pre_process_av" (id=2365752384816) was called 1 times


In [7]:
fa = feat.preprocess_faults(fa,remove_same_location_faults = True)

Running: Function "preprocess_faults" (id=2365752381936) was called 1 times
Running: Function "load_module_lookup" (id=2365752383088) was called 1 times
duplicated location faults removed - max duration kept


In [8]:
fa_floor = feat.floor_shift_time_fa(fa, shift=0)

Time shifted by 0Minutes


In [9]:
fa_sel = feat.fault_select(fa_floor, fault_select_options = {'Tote Colour':['Grey','Both']})
                                                            
fa_agg = feat.faults_aggregate(fa_sel,fault_agg_level= 'Asset Code', agg_type = 'count')

Running: Function "faults_aggregate" (id=2365752384096) was called 1 times


In [10]:
av_sel,at_sel = feat.av_at_select(av, at, remove_high_AT = False,availability_select_options = None)

av_agg = feat.aggregate_availability(av_sel, agg_level = 'Module')
at_agg = feat.aggregate_totes(at_sel, agg_level = 'Module')

#at_agg = at_agg[at_agg['TOTES']>100]

Running: Function "aggregate_availability" (id=2365752561872) was called 1 times


In [11]:
df = feat.merge_av_fa_at(av_agg ,at_df=at_agg, fa_df = fa_agg ,faults=True, totes = True, agg_level = 'Module',remove_0=True)

Datasets merged


In [12]:
X,y = md.gen_feat_var(df,target = 'Grey Tote Loss')
y=1-y

In [13]:
X_train, X_test, y_train, y_test = md.split(X,y,split_options = {'test_size': 0.3,
                                                                 'random_state': None})

RF_mdl,predictions_RF=md.run_RF_model(X_train, X_test, y_train, y_test,RF_options = {'num_trees': 100, 
                                                                                    'criterion': 'mse', 
                                                                                    'max_depth': None, 
                                                                                    'dtree': False})

Linear_mdl,predictions_LM,Coeff,fit_metrics = md.run_LR_model(X_train, X_test, y_train, y_test)

XGB_mdl,predictions_XGB,Importance_XGB = md.run_XGB_model(X_train, X_test, y_train, y_test, XGB_options = {'num_trees': 100,
                                                                                                         'max_depth': None})

selection_options= {'thres': -np.inf,
                    'max_feat': None}

X_sel_rf = md.select_features(X, y, RF_mdl, selection_options)

X_sel_lm = md.select_features(X, y, Linear_mdl, selection_options)

X_sel_XGB = md.select_features(X, y, XGB_mdl, selection_options)

                                                                                                           #X_train, X_test, y_train, y_test = md.split(X_sel_lm,y,test_size=0.3,random_state=101)

#Linear_mdl,predictions_LM,Coeff,fit_metrics = md.run_LR_model(X_train, X_test, y_train, y_test)

cv_R2 = md.cross_validate_r2(Linear_mdl, X_sel_lm, y)

cv_R2 = md.cross_validate_r2(RF_mdl,X_sel_rf, y)

cv_R2 = md.cross_validate_r2(XGB_mdl, X_sel_XGB, y)

#md.stats_model(Linear_mdl,X_test,y_test)

Feature Importance Ranking: 
 
    Importance      Feature
0    0.459242        TOTES
1    0.023793  C0503RDC101
2    0.019810  C1402STA061
3    0.018856          C10
4    0.016379  C1102STA071
5    0.015106          C08
6    0.014577  C1104STA197
7    0.013081       SCSM12
8    0.012199       SCSM04
9    0.012027          C14

Regression Metrics: 
 
           RF Metrics
MAE         0.081098
MSE         0.012094
RMSE        0.109973
MAPE%      21.690000
ACC%       78.310000
OOB         0.107401
R2_Train    0.879911
R2_Pred     0.091489 

Feature Importance Ranking: 
 
    Importance      Feature
0    0.063153  C1302STA080
1    0.062901  C1104STA181
2    0.046918  C1002STA097
3    0.036294  C0503RDC101
4    0.031880          C10
5    0.024633  C1101ACH001
6    0.023962  C1302STA061
7    0.023853  C0802STA077
8    0.021618  C1002STA076
9    0.020822  C1102STA071

Regression Metrics: 
 
          XGB Metrics
MAE        0.0836619
MSE        0.0132896
RMSE        0.115281
MAPE%          16

### Add LM - metrics

In [23]:
sc.lm_coefficients(Linear_mdl, X, y)

C:\Users\Jamie.williams\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1847: RuntimeWarning: invalid value encountered in greater_equal
  cond2 = (x >= np.asarray(_b)) & cond0


,Feature,Coefficients,Standard Errors,t values,Probabilities
0,Constant,0.8597,NaN,NaN,NaN
1,C05,0.0012,NaN,NaN,NaN
2,C0501STA002,0.0031,NaN,NaN,NaN
3,C0501STA012,-0.0049,NaN,NaN,NaN
4,C0501STA217,-0.0000,NaN,NaN,NaN
...,...,...,...,...,...
368,SCSM17,0.0005,NaN,NaN,NaN
369,SCSM18,0.0007,NaN,NaN,NaN
370,SCSM19,0.0000,NaN,NaN,NaN
371,SCSM20,0.0001,NaN,NaN,NaN


In [15]:
X_train = sm.add_constant(X_train)
model = sm.OLS(y_train,X_train)
results = model.fit()
print(results.summary())

X_new = sm.add_constant(X_test)
#y_prediction =  results.predict(X_test)

# plt.scatter(y_prediction,y_test)

# from sklearn.metrics import r2_score
# print(r2_score(y_prediction,y_test))

                            OLS Regression Results                            
Dep. Variable:         Grey Tote Loss   R-squared:                       0.275
Model:                            OLS   Adj. R-squared:                  0.212
Method:                 Least Squares   F-statistic:                     4.384
Date:                Mon, 14 Dec 2020   Prob (F-statistic):           1.14e-41
Time:                        18:41:38   Log-Likelihood:                 1496.0
No. Observations:                1498   AIC:                            -2752.
Df Residuals:                    1378   BIC:                            -2115.
Df Model:                         119                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const           0.2751      0.008     36.052      

C:\Users\Jamie.williams\Anaconda3\lib\site-packages\statsmodels\base\model.py:1362: RuntimeWarning: invalid value encountered in true_divide
  return self.params / self.bse
C:\Users\Jamie.williams\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1932: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)


In [16]:
df_greyhigh_ID = df.drop(['Grey Tote Loss','TOTES','timestamp'],axis=1)

df_greyhigh_ID.sum().sort_values(ascending=False).head(20)

SCSM18         26123.0
C09            24913.0
C12            24713.0
SCSM12         24027.0
C14            20911.0
C08            20450.0
C13            16342.0
SCSM03         15365.0
SCSM15         12730.0
C11            11993.0
C07            11958.0
C06            11169.0
SCSM04         10804.0
SCSM07         10802.0
SCSM11         10774.0
SCSM01         10616.0
SCSM20          9603.0
C1404STA164     8889.0
SCSM13          8384.0
C05             8238.0
dtype: float64

In [17]:
df_grey_ID = df.drop(['Grey Tote Loss','TOTES','timestamp'],axis=1)

df_grey_ID.sum().sort_values(ascending=False).head(20)

SCSM18         26123.0
C09            24913.0
C12            24713.0
SCSM12         24027.0
C14            20911.0
C08            20450.0
C13            16342.0
SCSM03         15365.0
SCSM15         12730.0
C11            11993.0
C07            11958.0
C06            11169.0
SCSM04         10804.0
SCSM07         10802.0
SCSM11         10774.0
SCSM01         10616.0
SCSM20          9603.0
C1404STA164     8889.0
SCSM13          8384.0
C05             8238.0
dtype: float64

In [18]:
df_greyhigh_Asset = df.drop(['Grey Tote Loss','TOTES','timestamp'],axis=1)

df_greyhigh_Asset.sum().sort_values(ascending=False).head(20)

SCSM18         26123.0
C09            24913.0
C12            24713.0
SCSM12         24027.0
C14            20911.0
C08            20450.0
C13            16342.0
SCSM03         15365.0
SCSM15         12730.0
C11            11993.0
C07            11958.0
C06            11169.0
SCSM04         10804.0
SCSM07         10802.0
SCSM11         10774.0
SCSM01         10616.0
SCSM20          9603.0
C1404STA164     8889.0
SCSM13          8384.0
C05             8238.0
dtype: float64

In [19]:
df_grey_Asset = df.drop(['Grey Tote Loss','TOTES','timestamp'],axis=1)

df_grey_Asset.sum().sort_values(ascending=False).head(20)

SCSM18         26123.0
C09            24913.0
C12            24713.0
SCSM12         24027.0
C14            20911.0
C08            20450.0
C13            16342.0
SCSM03         15365.0
SCSM15         12730.0
C11            11993.0
C07            11958.0
C06            11169.0
SCSM04         10804.0
SCSM07         10802.0
SCSM11         10774.0
SCSM01         10616.0
SCSM20          9603.0
C1404STA164     8889.0
SCSM13          8384.0
C05             8238.0
dtype: float64

In [20]:
scs_raw

,Number,Alert,Entry time,PLC,Desk,Duration,Fault ID
0,141254,C17: C1701STA254 254B12 occupancy sensor fault,01/11/2020 00:00,C17,P01,0.00:00:13,3416
1,138183,C17: C1701STA012 012B13 occupancy sensor fault,01/11/2020 00:00,C17,P01,0.00:00:03,3416
2,122518,C15: C1502STA034 034B12 occupancy sensor fault,01/11/2020 00:00,C15,P02,0.00:00:05,3416
3,444609,SCS M09: single module 1 emergency stop unit ...,01/11/2020 00:00,SCSM09,SCSM09,0.00:00:13,10408
4,139815,C17: C1701STA112 112B13 occupancy sensor fault,01/11/2020 00:00,C17,P01,0.00:00:08,3416
...,...,...,...,...,...,...,...
174085,439314,SCS228: sto: lifter occupation wrong in auto-m...,25/11/2020 09:03,SCSM22,SCS228,0.00:00:04,5113
174086,439314,SCS228: sto: lifter occupation wrong in auto-m...,25/11/2020 09:03,SCSM22,SCS228,0.00:00:08,5113
174087,439314,SCS228: sto: lifter occupation wrong in auto-m...,25/11/2020 09:04,SCSM22,SCS228,0.00:00:22,5113
174088,439308,SCS228: sto: locking device timeout close X:45...,25/11/2020 03:15,SCSM22,SCS228,0.00:03:29,5107
